In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/KalmanNet_Visual/KalmanNet_VO

/content/drive/MyDrive/KalmanNet_Visual/KalmanNet_VO


In [3]:
import os
import torch
import sys
from datetime import datetime
from Extended_sysmdl_visual import SystemModel
import numpy as np
from EKF_test_visual import EKFTest
#sys.path.insert(1, "Simulations/Pendulum/model.py")
from model import f, h_full, h_partial, h_nonlinear
import matplotlib.pyplot as plt
from KalmanNet_sysmdl import SystemModel
from KalmanNet_nn_NewArch_visual import KalmanNetNN
from Pipeline_KF_visual import Pipeline_KF

if torch.cuda.is_available():
    dev = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    dev = torch.device("cpu")
    print("Running on the CPU")

Running on GPU


In [6]:
NL_m = 2
NL_n = 2

m1_0 = torch.Tensor([[0.0], [0.0]]).to(dev)
m2_0 = torch.eye(NL_m).to(dev)
h = h_full

EPOCHS = 20
BATCH_SIZE = 10
LEARNING_RATE = 0.1
WEIGHT_DECAY = 1e-6
vs = [0, -10, -20]
vs= [0]
r2s = [10, 2, 1, 0.5, 0.1, 0.01, 0.001, 0.0001]
r2s = [0.1]
ROOT_PATH = "/content/drive/MyDrive/KalmanNet_Visual/KalmanNet_VO/"
#ROOT_PATH =  r"C:\Users\damis\Google Drive/"
PATH_SAVE_LOSS = ROOT_PATH + "Simulations_results/Pendulum/KalmanNet/loss/"
os.makedirs(PATH_SAVE_LOSS, exist_ok=True)
PATH_SAVE_MODEL = ROOT_PATH + "Simulations_results/Pendulum/KalmanNet/model/"
os.makedirs(PATH_SAVE_MODEL, exist_ok=True)

for v in vs:
  q2s = list(map(lambda x: x*(10**(v/10)), r2s))
  for r2, q2 in zip(r2s, q2s):
    q = np.sqrt(q2)
    r = np.sqrt(r2)
    Q = q*q*torch.eye(NL_m)
    R = r*r*torch.eye(NL_n)
    input_ = np.load(ROOT_PATH+f"Datasets/Pendulum/decimated_noisy_data/pendulum_decimated_noisy_q2_{q2:.0e}_r2_{r2:.0e}_v{v}.npz")
    target_ = np.load(ROOT_PATH+f"Datasets/Pendulum/decimated_clean_data/pendulum_decimated_q2_{q2:.0e}_v_{v}.npz")
    train_input = torch.from_numpy(input_["training_set"][:100, :, :]).float().to(dev)
    train_target = torch.from_numpy(target_["training_set"][:100, :, :]).float().to(dev)
    test_input = torch.from_numpy(input_["test_set"][:, :, :]).float().to(dev)
    test_target = torch.from_numpy(target_["test_set"]).float().to(dev)
    cv_input = torch.from_numpy(input_["validation_set"][:100, :, :]).float().to(dev)
    cv_target = torch.from_numpy(target_["validation_set"][:100, ...]).float().to(dev)

    traj_length_training = train_input.shape[2]
    traj_length_test = test_input.shape[2]
    today = datetime.today()
    now = datetime.now()
    strToday = today.strftime("%m.%d.%y")
    strNow = now.strftime("%H:%M:%S")
    strTime = strToday + "_" + strNow
    print("Current Time =", strTime)

    system_model = SystemModel(f=f, Q=Q, h=h, R=R, T=traj_length_training, T_test=traj_length_test, prior_Q=None, prior_Sigma=None, prior_S=None)
    system_model.InitSequence(m1_0, m2_0)
    KNet_Pipeline = Pipeline_KF(Time=strTime, folderName=PATH_SAVE_MODEL, modelName=f"KalmanNet_q2_{q2:.0e}_r2_{r2:.0e}_v_{v}", data_name="Pendulum")
    KNet_Pipeline.setssModel(system_model)

    KNet_model = KalmanNetNN()
    KNet_model.Build(system_model)
    KNet_Pipeline.setModel(KNet_model)

    KNet_Pipeline.setTrainingParams(fix_H_flag=1, n_Epochs=EPOCHS, n_Batch=BATCH_SIZE, learningRate=LEARNING_RATE, weightDecay=WEIGHT_DECAY)
    train_loss_list, val_loss_list = KNet_Pipeline.NNTrain(train_input=train_input, train_target=train_target, cv_input=cv_input, cv_target=cv_target, model_AE_conv=None, matrix_data_flag=1)
    #KNet_Pipeline.print_process(val_loss_list, train_loss_list, f"q2: {q2:.0e}, r2: {r2:.0e}, v: {v}")

    #MSE_test_linear_avg, MSE_test_theta_linear_avg, MSE_test_omega_linear_avg, MSE_test_dB_avg, prediction=KNet_Pipeline.NNTest(n_Test=test_input.shape[0], test_input=test_input, test_target=test_target, model_AE=None, model_AE_conv=None, matrix_data_flag=1)
    #np.savez(PATH_SAVE_LOSS,  MSE_avg_linear=MSE_test_linear_avg.cpu(), MSE_avg_linear_theta=MSE_test_theta_linear_avg.cpu(), MSE_avg_linear_omega=MSE_test_omega_linear_avg.cpu(), MSE_avg_dB= MSE_test_dB_avg.cpu(), prediction=prediction.cpu())





Current Time = 05.16.22_08:00:54
0 MSE Training : tensor(inf) [dB] MSE Validation : tensor(inf) [dB] , timing  1652687744.986189
Optimal idx: 0 Optimal : 1000 [dB]
1 MSE Training : tensor(nan) [dB] MSE Validation : tensor(nan) [dB] , timing  1652687838.395176
Optimal idx: 0 Optimal : 1000 [dB]


KeyboardInterrupt: ignored